In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
timesteps = 28    # timesteps
num_classes = 10    # MNIST total classes (0-9 digits) >> MNIST data has only 10 labels
num_input = 28    # MNIST data input (img shape: 28 * 28)

X = tf.placeholder(tf.float32, [None, timesteps, num_input]) # X size N * 28 * 28
Y = tf.placeholder(tf.float32, [None, num_classes]) # Y size N * 10

In [4]:
# Unstack to get a list of tensors of shape (batch_size, n_input)
X = tf.unstack(X, timesteps, 1)

#### LSTM cell

In [5]:
# Define a LSTM cell with TensorFlow
num_hidden = 128    # Hidden layer num of features
lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

#### LSTM network

In [6]:
# Get LSTM cell output
outputs, states = rnn.static_rnn(lstm_cell, X, dtype=tf.float32)

In [7]:
W = tf.Variable(tf.random_normal([num_hidden, num_classes]))    # W size is 128 * 10
b = tf.Variable(tf.random_normal([num_classes]))    # b size is 10

In [8]:
# Linear activation, using rnn inner loop last
logits = tf.matmul(outputs[-1], W) + b
prediction = tf.nn.softmax(logits)

In [9]:
# Define loss and optimizer
learning_rate = 0.001

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
loss_op = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss_op)

In [10]:
# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [11]:
# Initialize the variables (i.e. assign their default value)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

#### Optimization

In [12]:
def print_step(step, loss, acc):
    print("Step ", step,
          "\tMinibatch Loss {:.4f}".format(loss),
          "\tTraining Accuracy {:.3f}".format(acc))

In [13]:
batch_size = 128
training_steps = 1000

# step == epoch
for step in range(1, training_steps + 1):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    
    # Reshape data to get 28 seq of 28 elements
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
    
    # Run optimization op (backpropagation)
    sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
    
    if step % 100 == 0:
        loss, acc = sess.run([loss_op, accuracy],
                            feed_dict={X: batch_x, Y: batch_y})
        print_step(step, loss, acc)

print("Optimization Finished!")

TypeError: unhashable type: 'list'

#### Test

In [ ]:
# Calculate accuracy for 128 MNIST test images
reshape_size = (-1, timesteps, num_input)
test_len = 128
test_data = mnist.test.images[:test_len].reshape(reshape_size)
test_label = mnist.test.labels[:test_len]
print("Testing Accuracy ", sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))